In [ ]:
import pygame
import numpy as np
import time
from IPython.display import HTML
import random
import threading
import types
import traceback
from toggle_input import hide_toggle as hide
from collections import defaultdict 
def def_value(): 
    return None
stateManager = defaultdict(def_value) 
pixel = 64
playerXPosition = (width/2) - (pixel/2) 
playerYPosition = height - pixel - 10     
playerXPositionChange = 0

class GameThread(threading.Thread):
    def __init__(self,  *args, **kwargs):
        super(GameThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    
#Initial Config
class Game:
    def __init__(self,display_width,display_height,window_color,title):
        self.size = (display_width,display_height)
        self.title = title
        self.running = True
        self.scene = list()
        self.window_color=window_color
        self.event_handlers = dict()
        self.event_handlers[(('type',pygame.QUIT),)] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_q))] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_ESCAPE))] = self.on_quit
        self.flipdelay=16
        self.tickcounter=0
        
    def render(self):
        self.disp.fill((0,0,0))
        for obj in self.scene:
            try:
                obj.render(self.disp)
            except Exception:
                traceback.print_exc()
                self.scene.remove(obj)
                print("Rendering Error: Object "+str(obj)+" removed from the scene")
        # unlike .update() just updates the screen as a whole          
        pygame.display.flip()
    def update(self):
        dt=pygame.time.get_ticks()- self.tickcounter
        for obj in self.scene:
            try:
                obj.update(dt)
            except Exception:
                traceback.print_exc()
                self.scene.remove(obj)
                print("Update Error: Object "+str(obj)+" removed from the scene")
        self.tickcounter=pygame.time.get_ticks()
        pygame.time.delay(self.flipdelay)
    
    def on_quit(self, event):
        self.running = False
    
    def process_events(self):
        for event in pygame.event.get():
            dire = dir(event)
            for eh in self.event_handlers.keys():
                callit=True
                for (attrname,attrvalue) in eh:
                    if (not attrname in dire) or (event.__getattribute__(attrname)!=attrvalue):
                        callit=False
                        break
                if callit:
                    self.event_handlers[eh](event)
        
    def display(self):
        pygame.init()
        self.disp=pygame.display.set_mode(self.size, pygame.HWSURFACE | pygame.DOUBLEBUF)
        pygame.display.set_caption(self.title)
        self.clock=pygame.time.Clock()
        self.tickcounter=pygame.time.get_ticks()
        while( self.running ):
            try:
                self.render()
                self.process_events()
                self.update()
            except Exception:
                traceback.print_exc()
                pygame.time.delay(10000)
        pygame.quit()

def startGame(display_width=500,display_height=500,window_color=(0,0,0),title='NoteG Prototype'):
    global game,engine,scene
    game = Game(display_width,display_height,window_color,title)
    engine = GameThread(target = game.display)
    engine.start()

def getSize():
    return pygame.display.get_surface().get_size()
    
hide()